In [ ]:
#### Red Wall Replication II

In [ ]:
dataset_name = "W14_comb"
df_list = 'BES_Panel'
#            'BES_reduced_with_na'
#           ]#,"BESnumeric"]

%matplotlib inline 
%run BES_header.py {dataset_name} {df_list}

del BES_Panel

In [ ]:
from random import randint
import time

In [ ]:
def add_identity(axes, *line_args, **line_kwargs):
    identity, = axes.plot([], [], *line_args, **line_kwargs)
    def callback(axes):
        low_x, high_x = axes.get_xlim()
        low_y, high_y = axes.get_ylim()
        low = max(low_x, low_y)
        high = min(high_x, high_y)
        identity.set_data([low, high], [low, high])
    callback(axes)
    axes.callbacks.connect('xlim_changed', callback)
    axes.callbacks.connect('ylim_changed', callback)
    return axes


def sm_and_xgb_plot(var_stub,title,specific_vars,df_simp,geomask=None,window=None,dependence_plots=False):
    from scipy import stats
    Treatment = var_stub
    var_list = [var_stub]
    var_stub_list = [var_stub,]
    df_simp[var_stub] = BES_census_data[var_stub]
    mask = df_simp[var_stub].notnull()
    if geomask is not None:
        mask = mask&geomask
    
    min_features = 30
    colname = var_stub
#     dependence_plots = True

    alg = get_xgboost_alg(classification_problem=False)
    (explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
        xgboost_run(subdir=colname,dataset=df_simp[mask][specific_vars+[var_stub]],
                var_list=var_list,var_stub_list=var_stub_list,
                use_specific_weights=None,
                min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
                skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
                title = title)
    xgboost_pred = pd.Series(alg.predict(df_simp[train_columns]),index=df_simp.index)

    import statsmodels.api as sm
    Y = df_simp[mask][var_stub]
    ## default mean imputation!
    X = df_simp[mask][specific_vars].astype('float').fillna(df_simp[mask][specific_vars].mean())
#     X = sm.add_constant(X)
    X["const"]=1.0
    model = sm.OLS(Y,X)
    results = model.fit()
    Xnew = df_simp[train_columns].astype('float')
#     Xnew = sm.add_constant(Xnew)
    Xnew["const"]=1.0
    statsmodels_pred = model.predict(results.params,Xnew)
    display( results.summary() );
    #########################################################

    ons_ids = BES_census_data.loc[df_simp.index,"ONSConstID"]
#     if geomask is not None:
#         ons_ids = ons_ids[geomask]

    xgb_var = title+"_xgb"+"_und"
    sm_var = title+"_sm"+"_und"

#     print("!!!!!!!!!!!!!!!!!!!!!!")
#     print(xgb_var,sm_var,title)
    merged[title+"_xgb"]=np.nan
    merged.loc[ ons_ids , title+"_xgb" ] = xgboost_pred.values
    merged[xgb_var] = merged[var_stub] - merged[title+"_xgb"]

    merged[title+"_sm"]=np.nan
    merged.loc[ ons_ids , title+"_sm" ] = statsmodels_pred
    merged[sm_var] = merged[var_stub] - merged[title+"_sm"]

    merged_mask = merged.reset_index()['PCON13CD'].isin(ons_ids[mask]).values
    
    

    fig, axes = plt.subplots(nrows=1,ncols=5,figsize=(16.5,4))
    merged[xgb_var][merged_mask].hist(bins=50,ax=axes[0])
#     mask = merged[xgb_var].notnull()
    sns.regplot(merged[title+"_xgb"][merged_mask],merged[var_stub][merged_mask],ax=axes[1])
    r, p, n= corr_simple_pearsonr(merged[title+"_xgb"],merged[var_stub], mask=merged_mask, round_places=2)
    axes[1].text(0.05, .9, 'r={:.2f}, p={:.2g}'.format(r, round(p,4)),verticalalignment='top', transform=axes[1].transAxes);
    add_identity(axes[1], color='r', ls='--')

    merged[sm_var][merged_mask].hist(bins=50,ax=axes[2])
#     mask = merged[sm_var].notnull()
    sns.regplot(merged[title+"_sm"][merged_mask],merged[var_stub][merged_mask],ax=axes[3])
    corr_mask = merged[title+"_sm"][merged_mask].notnull()&merged[var_stub][merged_mask].notnull()
    r, p, n= corr_simple_pearsonr(merged[title+"_sm"],merged[var_stub], mask=merged_mask, round_places=2)
#     r, p = stats.pearsonr(  merged[title+"_sm"][merged_mask][corr_mask]merged[var_stub][merged_mask][corr_mask] )
    axes[3].text(0.05, .9, 'r={:.2f}, p={:.2g}'.format(r, round(p,4)),verticalalignment='top', transform=axes[3].transAxes);
    add_identity(axes[3], color='r', ls='--')

    axes[0].set_ylabel("freq");
    axes[0].set_xlabel("underperf vs model (xgb)");
    axes[1].set_ylabel(var_stub);
    axes[1].set_xlabel(var_stub+" pred (xgb)");
    axes[2].set_ylabel("freq");
    axes[2].set_xlabel("underperf vs model (sm)");
    axes[3].set_ylabel(var_stub);
    axes[3].set_xlabel(var_stub+" pred (sm)");

    
    g = sns.regplot(x=merged[xgb_var][merged_mask], y=merged[sm_var][merged_mask], ax = axes[4])
#     r, p = stats.pearsonr( merged[xgb_var][merged_mask] , merged[sm_var][merged_mask] )
    r, p, n= corr_simple_pearsonr(merged[xgb_var],merged[sm_var], mask=merged_mask, round_places=2)
    axes[4].text(0.05, .9, 'r={:.2f}, p={:.2g}'.format(r, round(p,4)),verticalalignment='top', transform=axes[4].transAxes);
    axes[4].set_ylabel("underperf vs model (sm)");
    axes[4].set_xlabel("underperf vs model (xgb)");
    add_identity(axes[4], color='r', ls='--')
    plt.suptitle("Residual histograms and regression plots for each of Xgboost/Statsmodels models\nFollowed by regression of each models residuals");
    ##########################################################

    top75 = False
    add_colorbar = True

    if top75 == True:
        merged[xgb_var] = merged[xgb_var]<merged[xgb_var].quantile(.25)
        merged[sm_var] = merged[sm_var]<merged[sm_var].quantile(.25)
    else:
        merged[xgb_var] = -merged[xgb_var]
        merged[sm_var] = -merged[sm_var]
    if window is not None:
        merged_window = merged.reset_index()['PCON13CD'].isin(ons_ids[window]).values
        merged.loc[~merged_window,xgb_var]=np.nan
        merged.loc[~merged_window,sm_var]=np.nan
    else:
        merged.loc[~merged_mask,xgb_var]=np.nan
        merged.loc[~merged_mask,sm_var]=np.nan
              

    suptitle = "2017GE: Conservative Performance\nRelative To Demographic Model"
    symmetrical_colour_scale = True
    # set the range for the choropleth
    vmin, vmax = np.floor(merged[[xgb_var,sm_var]].min().min()/10)*10, np.ceil(merged[[xgb_var,sm_var]].max().max()/10)*10
    
    if symmetrical_colour_scale:
        max_abs_scale = max(abs(vmin),abs(vmax))
        vmin = (vmin/abs(vmin))*max_abs_scale
        vmax = (vmax/abs(vmax))*max_abs_scale

    # create figure and axes for Matplotlib
    fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(16, 10))

    # create map
    merged.plot(column=xgb_var, cmap='seismic', norm=plt.Normalize(vmin=vmin, vmax=vmax), linewidth=0.1, ax=ax[0], edgecolor='0.8',aspect=1)
    # remove the axis
    ax[0].axis('off')
    # add a title
    ax[0].set_title(suptitle+"(xgb)", \
                  fontdict={'fontsize': '18',
                            'fontweight' : '3'})

    # create map
    merged.plot(column=sm_var,  cmap='seismic', norm=plt.Normalize(vmin=vmin, vmax=vmax), linewidth=0.1, ax=ax[1], edgecolor='0.8',aspect=1)
    # remove the axis
    ax[1].axis('off')
    # add a title
    ax[1].set_title(suptitle+"(sm)", \
                  fontdict={'fontsize': '18',
                            'fontweight' : '3'})



    # create an annotation for the  data source
    plt.annotate('Source: https://www.britishelectionstudy.com/data-objects/linked-data/',
               xy=(0.3, .17), xycoords='figure fraction',
               verticalalignment='bottom',
                 #horizontalalignment='middle', 
               fontsize=10, color='#555555')

    if add_colorbar:
        # Create colorbar as a legend
        sm = plt.cm.ScalarMappable(cmap=plt.cm.get_cmap('seismic').reversed(),#'seismic',
                                   norm=plt.Normalize(vmin=vmin, vmax=vmax))
#         sm._A = []
        cbar = fig.colorbar(sm,
#                             pad=-.5,
#                             orientation='horizontal',
                            pad=0,
                            location='top',ax=ax[0:2], shrink=.2,fraction=.3)
        # location='bottom',

    Treatment = "Chloropleth"
    output_subfolder = create_subdir(BES_output_folder,Treatment)
    fname = output_subfolder +suptitle +".png"
    # this will save the figure as a high-res png. you can also save as svg
    plt.savefig( fname, bbox_inches='tight', dpi=1000 )
    return ()

def model_comparison(var_stub,title,specific_vars_list,df_simp,geomask=None,model_type='sm',window=None,dependence_plots=False):
    from scipy import stats
    Treatment = var_stub
    var_list = [var_stub]
    var_stub_list = [var_stub,]
    df_simp[var_stub] = BES_census_data[var_stub]
    mask = df_simp[var_stub].notnull()
    if geomask is not None:
        mask = mask&geomask
    ons_ids = BES_census_data.loc[df_simp.index,"ONSConstID"]    
#     print("MASK DEBUG")
#     print(mask.sum())
    ######
    count=1
#     model_list = []
    for specific_vars in specific_vars_list:
        print("model"+str(count))
        min_features = 30
        colname = var_stub
#         dependence_plots = True
        alg = get_xgboost_alg(classification_problem=False)
        (explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
            xgboost_run(subdir=colname,dataset=df_simp[mask][specific_vars+[var_stub]],
                    var_list=var_list,var_stub_list=var_stub_list,
                    use_specific_weights=None,
                    min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
                    skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
                    title = title)
        xgboost_pred = pd.Series(alg.predict(df_simp[train_columns]),index=df_simp.index)
        ######
        import statsmodels.api as sm
        Y = df_simp[mask][var_stub]
        X = df_simp[mask][specific_vars].astype('float').fillna(df_simp[mask][specific_vars].mean())
#         X = sm.add_constant(X)
        X["const"]=1.0
        model = sm.OLS(Y,X)
        results = model.fit()
        Xnew = df_simp[train_columns].astype('float')
#         Xnew = sm.add_constant(Xnew)
        Xnew["const"]=1.0
        statsmodels_pred = model.predict(results.params,Xnew)
        display( results.summary() );
        ######
        pred_name = title+"_model"+str(count)
        merged[pred_name]=np.nan
        if model_type == "xgb":
            merged.loc[ ons_ids , pred_name ] = xgboost_pred.values
        else:
            merged.loc[ ons_ids , pred_name ] = statsmodels_pred
        merged[pred_name+"_und"] = merged[var_stub] - merged[pred_name]
        
        count=count+1

    
    merged_mask = merged.reset_index()['PCON13CD'].isin(ons_ids[mask]).values
    

    if len(specific_vars_list)==2:
    
        fig, axes = plt.subplots(nrows=1,ncols=5,figsize=(16.5,4))
        merged[title+"_model1"+"_und"][merged_mask].hist(bins=50,ax=axes[0])
    #     mask = merged[xgb_var].notnull()
#         model1 = title+"_model1"
#         model2 = title+"_model2"
        sns.regplot(merged[title+"_model1"][merged_mask],merged[var_stub][merged_mask],ax=axes[1])
#         r, p = stats.pearsonr( merged[title+"_model1"][merged_mask] , merged[var_stub][merged_mask] )
        r, p, n= corr_simple_pearsonr(merged[title+"_model1"],merged[var_stub], mask=merged_mask, round_places=2)
        axes[1].text(0.05, .9, 'r={:.2f}, p={:.2g}'.format(r, round(p,4)),verticalalignment='top', transform=axes[1].transAxes);
        add_identity(axes[1], color='r', ls='--')
        
        merged[title+"_model2"+"_und"][merged_mask].hist(bins=50,ax=axes[2])
    #     mask = merged[sm_var].notnull()
        sns.regplot(merged[title+"_model2"][merged_mask],merged[var_stub][merged_mask],ax=axes[3])
#         r, p = stats.pearsonr( merged[title+"_model2"][merged_mask] , merged[var_stub][merged_mask] )
        r, p, n= corr_simple_pearsonr(merged[title+"_model2"],merged[var_stub], mask=merged_mask, round_places=2)
        axes[3].text(0.05, .9, 'r={:.2f}, p={:.2g}'.format(r, round(p,4)),verticalalignment='top', transform=axes[3].transAxes);
        add_identity(axes[3], color='r', ls='--')

        axes[0].set_ylabel("freq");
        axes[0].set_xlabel("model 1 residuals");
        axes[1].set_ylabel(var_stub);
        axes[1].set_xlabel(var_stub+" model 1 pred");
        axes[2].set_ylabel("freq");
        axes[2].set_xlabel("model 2 residuals");
        axes[3].set_ylabel(var_stub);
        axes[3].set_xlabel(var_stub+" model 2 pred");

        g = sns.regplot(x=merged[title+"_model1"+"_und"][merged_mask],
                        y=merged[title+"_model2"+"_und"][merged_mask], ax = axes[4])
#         r, p = stats.pearsonr( merged[title+"_model1"+"_und"][merged_mask],
#                                merged[title+"_model2"+"_und"][merged_mask] )
        r, p, n= corr_simple_pearsonr(merged[title+"_model1"],merged[title+"_model2"+"_und"], mask=merged_mask, round_places=2)
        axes[4].set_ylabel("model 1 residuals");
        axes[4].set_xlabel("model 2 residuals");
        axes[4].text(0.05, .9, 'r={:.2f}, p={:.2g}'.format(r, round(p,4)),verticalalignment='top', transform=axes[4].transAxes);
        add_identity(axes[4], color='r', ls='--')
        
        fig.tight_layout()
#         plt.suptitle("Residual histograms and regression plots for each of Xgboost/Statsmodels models\nFollowed by regression of each models residuals");
    ##########################################################

    top75 = False
    fraction = .25
    add_colorbar = True
#     plt_title = "2017GE: Conservative Performance\nRelative To Demographic Model"
    symmetrical_colour_scale = True    
    
    var_list = [title+"_model"+str(x)+"_und" for x in range(1,len(specific_vars_list)+1)]
    
    for resid in var_list:
        if top75 == True:
            merged[resid] = merged[resid]<merged[resid].quantile(fraction)
        else:
            merged[resid] = -merged[resid]
        if window is not None:
            merged_window = merged.reset_index()['PCON13CD'].isin(ons_ids[window]).values
            merged.loc[~merged_window,resid]=np.nan
        else:
            merged.loc[~merged_mask,resid]=np.nan



    # set the range for the choropleth
    vmin, vmax = np.floor(merged[var_list].min().min()/10)*10, np.ceil(merged[var_list].max().max()/10)*10
    if symmetrical_colour_scale:
        max_abs_scale = max(abs(vmin),abs(vmax))
        vmin = (vmin/abs(vmin))*max_abs_scale
        vmax = (vmax/abs(vmax))*max_abs_scale

    # create figure and axes for Matplotlib
    fig, ax = plt.subplots(nrows=1,ncols=len(specific_vars_list),
                           figsize=(16, 10))
    
    for var_no in range(len(var_list)):
        # create map
        merged.plot(column=var_list[var_no], cmap='seismic', norm=plt.Normalize(vmin=vmin, vmax=vmax), linewidth=0.1,
                    ax=ax[var_no], edgecolor='0.8',aspect=1)

        # remove the axis
        ax[var_no].axis('off')
        # add a title
        ax[var_no].set_title("model"+str(var_no+1), \
                      fontdict={'fontsize': '18',
                                'fontweight' : '3'})
    
#     # create map
#     merged.plot(column=sm_var,  cmap='seismic', norm=plt.Normalize(vmin=vmin, vmax=vmax), linewidth=0.1, ax=ax[1], edgecolor='0.8',aspect=1)
#     # remove the axis
#     ax[1].axis('off')
#     # add a title
#     ax[1].set_title(title+"(sm)", \
#                   fontdict={'fontsize': '18',
#                             'fontweight' : '3'})



    # create an annotation for the  data source
    plt.annotate('Source: https://www.britishelectionstudy.com/data-objects/linked-data/',
               xy=(0.3, .17), xycoords='figure fraction',
               verticalalignment='bottom',
                 #horizontalalignment='middle', 
               fontsize=10, color='#555555')

    if add_colorbar:
        # Create colorbar as a legend
        sm = plt.cm.ScalarMappable(cmap=plt.cm.get_cmap('seismic').reversed(),#'seismic',
                                   norm=plt.Normalize(vmin=vmin, vmax=vmax))
#         sm._A = []
        if len(specific_vars_list)==2:
            location = 'top'
        else:
            location = 'bottom'
        cbar = fig.colorbar(sm,
#                             pad=-.5,
#                             orientation='horizontal',
                            pad=0,
                            location=location,ax=ax[0:len(specific_vars_list)], shrink=.2,fraction=.3)
        # location='bottom',

#     plt.suptitle(title)
        
    Treatment = "Chloropleth"
    output_subfolder = create_subdir(BES_output_folder,Treatment)
    fname = output_subfolder +title +".png"
    # this will save the figure as a high-res png. you can also save as svg
    plt.savefig( fname, bbox_inches='tight', dpi=1000 )
    return ()

In [ ]:
BES_census_data_2019 = pd.read_stata( BES_small_data_files + "BES-2019-General-Election-results-file-v1.0.dta" )
print("BES_census_data 2019", BES_census_data_2019.shape )

In [ ]:
BES_census_data_2017 = pd.read_stata( BES_small_data_files + "BES-2017-General-Election-results-file-v1.0.dta" )
print("BES_census_data", BES_census_data_2017.shape )

In [ ]:
BES_census_data_2015 = pd.read_stata( BES_small_data_files + "BES-2015-General-Election-results-file-v2.21.dta" )
print("BES_census_data 2015", BES_census_data_2015.shape )

In [ ]:
BES_census_data = BES_census_data_2019.copy()

In [ ]:
#### BUG! -> check if in latest release, if yes, then tell them! ###########
BES_census_data.loc[BES_census_data["Country"].isin(["Scotland"]),"c11Households"] =\
    BES_census_data.loc[BES_census_data["Country"].isin(["Scotland"]),"c11Households"]*100
BES_census_data["ConstituencyName"] = BES_census_data["ConstituencyName"].replace("Ynys Môn","Ynys Mon")
BES_census_data["Winner15"] = BES_census_data["Winner15"].replace("Speaker.","Speaker")

In [ ]:
party_19_list = search(BES_census_data,"^(Con|Lab|Brexit|LD|UKIP|SNP|PC|Green)19$").index
BES_census_data[[x+"_elec" for x in party_19_list]] = BES_census_data[party_19_list].apply(lambda x: x*BES_census_data["Turnout19"]/100).fillna(0)
BES_census_data["DNV19_elec"] = 100-BES_census_data["Turnout19"]
BES_census_data["DNV&OTH19_elec"] = 100-BES_census_data["Turnout19"]-BES_census_data["Other19"].fillna(0)
BES_census_data["Auth_Right19_elec"] = BES_census_data["Con19_elec"]+BES_census_data["UKIP19_elec"]+BES_census_data["Brexit19_elec"]
BES_census_data["Lib_Left19_elec"] = BES_census_data["Lab19_elec"]+BES_census_data["LD19_elec"]+BES_census_data["PC19_elec"]+BES_census_data["SNP19_elec"]+BES_census_data["Green19_elec"]

party_17_list = search(BES_census_data,"^(Con|Lab|Brexit|LD|UKIP|SNP|PC|Green)17$").index
BES_census_data[[x+"_elec" for x in party_17_list]] = BES_census_data[party_17_list].apply(lambda x: x*BES_census_data["Turnout17"]/100).fillna(0)
BES_census_data["DNV17_elec"] = 100-BES_census_data["Turnout17"]
BES_census_data["DNV&OTH17_elec"] = 100-BES_census_data["Turnout17"]-BES_census_data["Other17"].fillna(0)
BES_census_data["Auth_Right17_elec"] = BES_census_data["Con17_elec"]+BES_census_data["UKIP17_elec"]#+BES_census_data["Brexit17_elec"]
BES_census_data["Lib_Left17_elec"] = BES_census_data["Lab17_elec"]+BES_census_data["LD17_elec"]+BES_census_data["PC17_elec"]\
                                    +BES_census_data["SNP17_elec"]+BES_census_data["Green17_elec"]

party_15_list = search(BES_census_data,"^(Con|Lab|Brexit|LD|UKIP|SNP|PC|Green)15$").index
BES_census_data[[x+"_elec" for x in party_15_list]] = BES_census_data[party_15_list]\
                                    .apply(lambda x: x*BES_census_data["Turnout15"]/100).fillna(0)
BES_census_data["DNV15_elec"] = 100-BES_census_data["Turnout15"]
BES_census_data["DNV&OTH15_elec"] = 100-BES_census_data["Turnout15"]-BES_census_data["Other15"].fillna(0)
BES_census_data["Auth_Right15_elec"] = BES_census_data["Con15_elec"]+BES_census_data["UKIP15_elec"]#+BES_census_data["Brexit17_elec"]
BES_census_data["Lib_Left15_elec"] = BES_census_data["Lab15_elec"]+BES_census_data["LD15_elec"]+BES_census_data["PC15_elec"]\
                                    +BES_census_data["SNP15_elec"]+BES_census_data["Green15_elec"]

party_10_list = search(BES_census_data,"^(Con|Lab|BNP|LD|UKIP|SNP|PC|Green)10$").index
BES_census_data[[x+"_elec" for x in party_10_list]] = BES_census_data[party_10_list].apply(lambda x: x*BES_census_data["Turnout10"]/100).fillna(0)
BES_census_data["DNV10_elec"] = 100-BES_census_data["Turnout10"]
BES_census_data["DNV&OTH10_elec"] = 100-(BES_census_data[party_10_list].apply(lambda x: x*BES_census_data["Turnout10"]/100).sum(axis=1))
BES_census_data["Auth_Right10_elec"] = BES_census_data["Con10_elec"]+BES_census_data["UKIP10_elec"]+BES_census_data["BNP10_elec"]
BES_census_data["Lib_Left10_elec"] = BES_census_data["Lab10_elec"]+BES_census_data["LD10_elec"]+BES_census_data["PC10_elec"]\
                                    +BES_census_data["SNP10_elec"]+BES_census_data["Green10_elec"]

party_05_list = search(BES_census_data,"^(Con|Lab|BNP|LD|UKIP|SNP|PC|Green)05$").index
BES_census_data[[x+"_elec" for x in party_05_list]] = BES_census_data[party_05_list].apply(lambda x: x*BES_census_data["Turnout05"]/100).fillna(0)
BES_census_data["DNV05_elec"] = 100-BES_census_data["Turnout05"]
BES_census_data["DNV&OTH05_elec"] = 100-BES_census_data["Turnout05"]-BES_census_data["Other05"].fillna(0)
BES_census_data["Auth_Right05_elec"] = BES_census_data["Con05_elec"]+BES_census_data["UKIP05_elec"]+BES_census_data["BNP05_elec"]
BES_census_data["Lib_Left05_elec"] = BES_census_data["Lab05_elec"]+BES_census_data["LD05_elec"]+BES_census_data["PC05_elec"]\
                                    +BES_census_data["SNP05_elec"]+BES_census_data["Green05_elec"]


In [ ]:
BES_census_data[search(BES_census_data_2015,"SpendPercent").index] = BES_census_data_2015[search(BES_census_data_2015,"SpendPercent").index]

BES_census_data[['SNPLongSpendPercent', 'SNPShortSpendPercent',
       'PCLongSpendPercent', 'PCShortSpendPercent']] = BES_census_data[['SNPLongSpendPercent', 'SNPShortSpendPercent',
       'PCLongSpendPercent', 'PCShortSpendPercent']].replace(".",np.nan).astype('float')

In [ ]:
# search(BES_census_data_2019,"05")

In [ ]:
occ_df = pd.read_excel( open(BES_small_data_files+'Census2011-ConstituencyProfile-uk.xlsx','rb'), sheet_name='KS608D' )
occ_df.columns = occ_df.loc[3]
occ_df = occ_df.drop([0,1,2,3,654,])
# occ_df

var_list = ["CON%ELEM","CON%OPS","CON%SAL_SERV","CON%CAR_LEI_","CON%SKILL","CON%ADMIN_SEC","CON%ASSOC_PROF_TECH",
 "CON%PROF","CON%MAN_DIR_SEN"]

# occ_df = occ_df.set_index("ONSConstID").loc[merged.index]
# var_list = ["CON%ELEM","CON%OPS","CON%SAL_SERV","CON%CAR_LEI_","CON%SKILL","CON%ADMIN_SEC","CON%ASSOC_PROF_TECH",
#  "CON%PROF","CON%MAN_DIR_SEN"]

BES_census_data[var_list] = occ_df.set_index("ONSConstID").loc[BES_census_data["ONSConstID"]][var_list].values

merseyside = ["Birkenhead","Bootle","Garston and Halewood","Knowsley",
             "Liverpool, Riverside","Liverpool, Walton","Liverpool, Wavertree","Liverpool, West Derby",
             "Sefton Central","Southport","St Helens North","St Helens South and Whiston",
             "Wallasey","Wirral South","Wirral West"]

BES_census_data["Merseyside"] = BES_census_data["ConstituencyName"].isin(merseyside).astype('float')

In [ ]:
welsh_df = pd.read_excel( open(BES_small_data_files+'welsh_constituency_data.xls','rb'), sheet_name='WelshLanguage' )
welsh_df.columns = welsh_df.loc[4]
welsh_df = welsh_df.drop([0,1,2,3,4,50,51,52])
welsh_df = welsh_df.replace("Ynys Môn","Ynys Mon")
welsh_df = welsh_df.set_index("Assembly Constituency Area")
BES_census_data["speakWelsh"] = [welsh_df.loc[x,"Percentage aged 3+ who can speak Welsh"] if x in welsh_df.index else 0 for x in BES_census_data["ConstituencyName"]  ]

In [ ]:
BES_census_data["c11DeprivedMean"] = (BES_census_data["c11Deprived1"]+(BES_census_data["c11Deprived2"]*2)+(BES_census_data["c11Deprived3"]*3)+(BES_census_data["c11Deprived4"]*4))/100
BES_census_data["c11CarsMean"] = (BES_census_data["c11CarsOne"]+(BES_census_data["c11CarsTwo"]*2)+(BES_census_data["c11CarsThree"]*3)+(BES_census_data["c11CarsFour"]*4))/100
BES_census_data["c11AgeMean"] = (
                                 BES_census_data["c11Age18to19"]*18.5+\
                                 BES_census_data["c11Age20to24"]*22+\
                                 BES_census_data["c11Age25to29"]*27+\
                                 BES_census_data["c11Age30to44"]*37+\
                                 BES_census_data["c11Age45to59"]*52+\
                                 BES_census_data["c11Age60to64"]*62+\
                                 BES_census_data["c11Age65to74"]*69.5+\
                                 BES_census_data["c11Age75to84"]*79.5+\
                                 BES_census_data["c11Age85to89"]*87+\
                                 BES_census_data["c11Age90plus"]*95  )/100
BES_census_data["c11DeprivedHigh"] =BES_census_data["c11Deprived3"]+BES_census_data["c11Deprived4"]
BES_census_data["c11DeprivedLow"] =BES_census_data["c11Deprived2"]+BES_census_data["c11Deprived1"]+BES_census_data["c11DeprivedNone"]
## kids
BES_census_data["c11Kids"] =BES_census_data["c11Age0to4"]+\
                            BES_census_data["c11Age5to7"]+\
                            BES_census_data["c11Age8to9"]+\
                            BES_census_data["c11Age10to14"]+\
                            BES_census_data["c11Age15"]+\
                            BES_census_data["c11Age16to17"]
# 'c11Age0to4', 'c11Age5to7', 'c11Age8to9', 'c11Age10to14', 'c11Age15',  'c11Age16to17',
## young adults
BES_census_data["c11YoungAdults"] =BES_census_data["c11Age18to19"]+\
                            BES_census_data["c11Age20to24"]+\
                            BES_census_data["c11Age25to29"]
# 'c11Age18to19', 'c11Age20to24', 'c11Age25to29',
## adult
BES_census_data["c11Adults"] =BES_census_data["c11Age30to44"]+\
                              BES_census_data["c11Age45to59"]
# 'c11Age30to44', 'c11Age45to59',
## elderly
BES_census_data["c11Elderly"] =BES_census_data["c11Age60to64"]+\
                            BES_census_data["c11Age65to74"]+\
                            BES_census_data["c11Age75to84"]+\
                            BES_census_data["c11Age85to89"]+\
                            BES_census_data["c11Age90plus"]
# 'c11Age60to64', 'c11Age65to74', 'c11Age75to84', 'c11Age85to89', 'c11Age90plus'
BES_census_data["c11HealthMean"] = (BES_census_data["c11HealthVeryBad"]+\
                                    BES_census_data["c11HealthBad"])-\
                                    (BES_census_data["c11HealthFair"]+\
                                    BES_census_data["c11HealthGood"])
                                    ## still not very good
BES_census_data["c11HealthVeryBad&Bad"] = BES_census_data["c11HealthVeryBad"]+BES_census_data["c11HealthBad"]
BES_census_data["c11HealthFair&Good"] = BES_census_data["c11HealthFair"]+BES_census_data["c11HealthGood"]
BES_census_data["c11CarsMoreThanNone"] = BES_census_data["c11CarsOne"]+BES_census_data["c11CarsTwo"]+BES_census_data["c11CarsThree"]+BES_census_data["c11CarsFour"]


In [ ]:
df = pd.read_csv(BES_small_data_files+"CentreForTowns"+os.sep+"pcon-classification-csv.csv")

# https://www.centrefortowns.org/our-towns
    
# Type	Definition	Number
# Villages (less than 5,000)	Places with less than 5,000 residents	5,568
# Communities (5-10k)	Places with between 5,000 and 10,000 residents	567
# Small towns (10k-30k)	Towns with between 10,000 and 30,000 residents	550
# Medium towns (30k – 75k)	Towns with between 30,000 and 75,000 residents	242
# Large towns (over 75k)	Towns with over 75,000 residents	102
# Core Cities	Core cities as defined by Pike et al (2016)	12

## doesn't seem so consisent - where is 'communities!'
replace_dict = {"Village or Smaller":"Village or smaller",}
df["classification"] = df["classification"].replace(replace_dict)
centre_for_towns = df.pivot(index='constituency_code', columns='classification', values='percent_of_constituency').fillna(0)
BES_census_data[centre_for_towns.columns] = centre_for_towns.loc[BES_census_data["ONSConstID"]].values

In [ ]:
parl_con_aggs = pd.read_csv(BES_small_data_files+"parliamentary constituency data"+os.sep+"Lots of aggregated parliamentary data.csv")

parl_con_aggs = parl_con_aggs.set_index("ONSConstID").drop(['Unnamed: 0', 'pano',
 'RESULTS2019_OTHER', 'RESULTS2019_Registered Voters', 'YOUGOVMRP_DEC_Con', 'YOUGOVMRP_DEC_Lab', 'YOUGOVMRP_DEC_LD',
       'YOUGOVMRP_DEC_Brexit', 'YOUGOVMRP_DEC_Green', 'YOUGOVMRP_DEC_SNP',
       'YOUGOVMRP_DEC_PC', 'YOUGOVMRP_DEC_Other', 'YOUGOVMRP_NOV_Con',
       'YOUGOVMRP_NOV_Lab', 'YOUGOVMRP_NOV_LD', 'YOUGOVMRP_NOV_Brexit',
       'YOUGOVMRP_NOV_Green', 'YOUGOVMRP_NOV_SNP', 'YOUGOVMRP_NOV_PC',
       'YOUGOVMRP_NOV_Other', 'ConVote17', 'TotalVote17', 'RejectedVote17',
       'Electorate17', 'ConVote15', 'TotalVote15', 'RejectedVote15',
       'Electorate15', 'ConVote10', 'TotalVote10', 'Electorate10',
       'c11Population','RESULTS2019_CON', 'RESULTS2019_LAB',
       'RESULTS2019_LIBDEM', 'RESULTS2019_GRN', 'RESULTS2019_SNP',
       'RESULTS2019_PC', 'RESULTS2019_BXP', 'RESULTS2019_UKIP',
       'RESULTS2019_Turnout','Turn10'],axis=1).select_dtypes(['float64','int64'])
parl_con_aggs = parl_con_aggs.drop(set(parl_con_aggs.columns).intersection(set(BES_census_data.columns)),axis=1)
parl_con_aggs = parl_con_aggs.loc[BES_census_data["ONSConstID"]]

BES_census_data[parl_con_aggs.columns] = parl_con_aggs.values
BES_census_data.columns = [x.replace("<","LT") for x in BES_census_data.columns]

In [ ]:
BES_constituency_aggregates = pd.read_csv(BES_small_data_files+"parliamentary constituency data"+os.sep+"BES_constituency_aggregates.csv")
BES_constituency_aggregates = BES_constituency_aggregates.set_index("ons_const_id").drop(["Unnamed: 0","constituency_name","pano"],axis=1).loc[BES_census_data["ONSConstID"]]
# (632, 6868)

## a lot - won't add before testing!

In [ ]:
nomis_dump = pd.read_csv(BES_data_folder+"Nomis"+os.sep+"2011ONS_England&Wales_by_parlcon.csv")
nomis_dump = nomis_dump.set_index("Unnamed: 0").loc[BES_census_data["ONSConstID"][~BES_census_data["Country"].isin(["Scotland"])]]
## England&Wales

In [ ]:
BES_census_data.shape

In [ ]:
# BES_census_data.columns = [x.replace("<","LT") for x in BES_census_data.columns]

In [ ]:
import geopandas as gpd
# set the filepath and load in a shapefile
fp = BES_small_data_files+"wpc.json"
map_df = gpd.read_file(fp)
# check data type so we can see that this is not a normal dataframe, but a GEOdataframe
# map_df.head()
# now let's preview what our map looks like with no data in it
# map_df.plot();
# join the geodataframe with the cleaned up csv dataframe
merged = map_df.set_index("PCON13CD").join(BES_census_data.set_index("ONSConstID"))
merged = merged.loc[BES_census_data["ONSConstID"]]
merged.head()

In [ ]:
drop_cols = ['Winner17', 'Winner15', 'Winner10','pano',#category
            'ConVote17', 'TotalVote17', 'Electorate17', 'ConVote15', 'TotalVote15',
       'Electorate15', 'ConVote10', 'TotalVote10', 'Electorate10', #int
             'LabVote19', 'Electorate19','RejectedVote19','LabVote05',
              'Lab17', 'LD17', 'SNP17', 'PC17', 'UKIP17', 'Green17',
             'Con0510', 'Lab0510', 'LD0510', 'SNP0510', 'PC0510',
       'BNP0510', 'UKIP0510', 'Green0510',
             'Con19', 'Lab19', 'LD19', 'SNP19', 'PC19', 'UKIP19', 'Green19',
       'Brexit19', 'Other19', 'Majority19', 'Turnout19', 'TotalVote19',
       'Con1719', 'Lab1719', 'LD1719', 'SNP1719', 'PC1719', 'UKIP1719',
       'Green1719',
             'ConVote19', 'LDVote19', 'SNPVote19', 'PCVote19', 'UKIPVote19',
       'GreenVote19', 'BrexitVote19',
             'leaveHanretty', 'remainHanretty',
             'Con05', 'Lab05', 'LD05', 'SNP05', 'PC05', 'UKIP05',
       'Green05', 'BNP05', 'Other05', 'Majority05', 'Turnout05', 'ConVote05',
       'LDVote05', 'SNPVote05', 'PCVote05', 'UKIPVote05', 'GreenVote05',
       'BNPVote05',
             'Winner05', 'ConPPCsex19', 'LabPPCsex19',
       'LDPPCsex19', 'SNPPPCsex19', 'PCPPCsex19', 'UKIPPPCsex19',
       'GreenPPCsex19', 'BrexitPPCsex19',
             # 'Con17',
       'Other17', 'Majority17', 'Turnout17', 'LabVote17', 'LDVote17',
       'SNPVote17', 'PCVote17', 'UKIPVote17', 'GreenVote17', 'Con1517',
       'Lab1517', 'LD1517', 'SNP1517', 'PC1517', 'UKIP1517', 'Green1517',
       'Con15', 'Lab15', 'LD15', 'SNP15', 'PC15', 'UKIP15', 'Green15',
       'Other15', 'Majority15', 'Turnout15', 'LabVote15', 'LDVote15',
       'SNPVote15', 'PCVote15', 'UKIPVote15', 'GreenVote15', 'BNPVote15',
       'Con1015', 'Lab1015', 'LD1015', 'SNP1015', 'PC1015', 'UKIP1015',
       'Green1015', 'Con10', 'Lab10', 'LD10', 'SNP10', 'PC10', 'UKIP10',
       'Green10', 'BNP10', 'Majority10', 'LabVote10', 'LDVote10',
       'SNPVote10', 'PCVote10', 'UKIPVote10', 'GreenVote10', 'BNPVote10',## float
       'ONSConstID', 'ConstituencyName', 'SeatChange1517', 'SeatChange1015',
       'ConPPC17', 'LabPPC17', 'LDPPC17', 'SNPPPC17', 'PCPPC17', 'UKIPPPC17',
       'GreenPPC17', 'ConPPC15', 'LabPPC15', 'LDPPC15', 'SNPPPC15', 'PCPPC15',
       'UKIPPPC15', 'GreenPPC15',     # object
       'ONSConstID', 'ConstituencyName', 'SeatChange1719', 'SeatChange1517',
       'SeatChange1015', 'ConPPC19', 'LabPPC19', 'LDPPC19', 'SNPPPC19',
       'PCPPC19', 'UKIPPPC19', 'GreenPPC19', 'BrexitPPC19', 'ConPPC17',
       'LabPPC17', 'LDPPC17', 'SNPPPC17', 'PCPPC17', 'UKIPPPC17', 'GreenPPC17',
       'ConPPC15', 'LabPPC15', 'LDPPC15', 'SNPPPC15', 'PCPPC15', 'UKIPPPC15',
       'GreenPPC15',
             'ConLongSpendPercent',
       'ConShortSpendPercent', 'LabLongSpendPercent', 'LabShortSpendPercent',
       'LDLongSpendPercent', 'LDShortSpendPercent', 'SNPLongSpendPercent',
       'SNPShortSpendPercent', 'PCLongSpendPercent', 'PCShortSpendPercent',
       'UKIPLongSpendPercent', 'UKIPShortSpendPercent',
       'GreenLongSpendPercent', 'GreenShortSpendPercent',
             'FOCALDATAMRP_DEC_Conservative',
 'FOCALDATAMRP_DEC_Labour',
 'FOCALDATAMRP_DEC_Liberal Democrats',
 'FOCALDATAMRP_DEC_SNP',
 'FOCALDATAMRP_DEC_Other',
 'FOCALDATAMRP_DEC_Green',
 'FOCALDATAMRP_DEC_Brexit Party',
 'FOCALDATAMRP_DEC_Plaid Cymru',
        'Winner19',
             'POLCOMP_Ec-Left-Right', 'POLCOMP_Soc-Lib-Con',
             'Turnout10',
            ]+['Con19_elec', 'Lab19_elec',
       'LD19_elec', 'SNP19_elec', 'PC19_elec', 'UKIP19_elec', 'Green19_elec',
       'Brexit19_elec', 'DNV19_elec', 'DNV&OTH19_elec', 'Auth_Right19_elec',
       'Lib_Left19_elec', 'Con17_elec', 'Lab17_elec', 'LD17_elec',
       'SNP17_elec', 'PC17_elec', 'UKIP17_elec', 'Green17_elec', 'DNV17_elec',
       'DNV&OTH17_elec', 'Auth_Right17_elec', 'Lib_Left17_elec', 'Con15_elec',
       'Lab15_elec', 'LD15_elec', 'SNP15_elec', 'PC15_elec', 'UKIP15_elec',
       'Green15_elec', 'DNV15_elec', 'DNV&OTH15_elec', 'Auth_Right15_elec',
       'Lib_Left15_elec', 'Con10_elec', 'Lab10_elec', 'LD10_elec',
       'SNP10_elec', 'PC10_elec', 'UKIP10_elec', 'Green10_elec', 'BNP10_elec',
       'DNV10_elec', 'DNV&OTH10_elec', 'Auth_Right10_elec', 'Lib_Left10_elec',
       'Con05_elec', 'Lab05_elec', 'LD05_elec', 'SNP05_elec', 'PC05_elec',
       'UKIP05_elec', 'Green05_elec', 'BNP05_elec', 'DNV05_elec',
       'DNV&OTH05_elec', 'Auth_Right05_elec', 'Lib_Left05_elec']+['TotalVote19',
                                                                  'TotalVote17', 'TotalVote15', 'TotalVote10',
       'TotalVote05','CLOCKFACEREP_diversity', 'CLOCKFACEREP_security',
       'CLOCKFACEREP_fecundity', 'CLOCKFACEREP_diversity_flatter',
       'CLOCKFACEREP_security_flatter']

# 'c11Population'

# df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1))

In [ ]:
## add to drop cols, based on variable type X
## decide how to setup merged/add variables to ... whichever dataset makes sense!  X
## add all the constructed variables X

In [ ]:
## add cft data X
## try BES_aggregates X
## try parl const aggregates X
## try nomis data X

## quick xgboost breakdown of each/all

In [ ]:
import pickle
def dim_red(df,n_components=None,red_type="Factor Analysis",show_first_x_comps=4):

    Treatment = "Census Data Decomposition"
    output_folder = "E:\\BES_analysis_data\\output\\CensusDataDecomposition\\"
    output_subfolder = output_folder + Treatment + os.sep
    if not os.path.exists( output_subfolder ):
        os.makedirs( output_subfolder )

    decomp_index = df.index
    decomp_columns = df.columns
    clean_feature_set_std = StandardScaler().fit_transform(df.values)
    decomp_std = pd.DataFrame(   clean_feature_set_std,
                                 columns = decomp_columns,
                                 index   = decomp_index      )
    if n_components is None:
        n_components = decomp_std.shape[1]
    
    # n_components = 4
    # (svd_solver='full', n_components='mle',whiten=True)
    
    if red_type == "Factor Analysis":
        decomp = FactorAnalysis(svd_method = 'lapack',n_components = n_components) ## ~10s ,n_components=30 -> 1.5 hr
    elif red_type == "PCA":
        decomp = PCA(n_components = n_components,svd_solver='full')    
    decomp_method = str(decomp).split("(")[0] 
    X_r = decomp.fit_transform(decomp_std)
    BES_decomp = pd.DataFrame(   X_r,
                                 columns = range(0,n_components),
                                 index   = decomp_index)

#     save = True # False => Load
#     if save & ( 'decomp' in globals() ): # SAVE    ##( 'decomp' not in globals() )
#         decomp_method = str(decomp).split("(")[0] 
#         subdir = output_subfolder + decomp_method
#         fname = subdir+ os.sep + decomp_method
    subdir = output_subfolder + decomp_method
    (BES_decomp, comp_labels, comp_dict) = display_components(n_components, decomp,
                                                              decomp_columns, BES_decomp, manifest=None, 
                                                              save_folder = subdir,  
                                                              show_first_x_comps= show_first_x_comps, show_histogram = False)    
    return (BES_decomp, comp_labels, comp_dict)

In [ ]:
results_df = pd.DataFrame(columns=["threshold","repeats","results"])

In [ ]:


# start = time.time()
# print("hello")
# end = time.time()
# print(end - start)

In [ ]:
%%time

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols+['Con17'],axis=1))[geomask]
df_simp[var_stub] = BES_census_data[var_stub][geomask].values
var_stub = "Con17_elec"

# results_list = []
# n_components=5
for n_comp in [5,10]:
    for corr_thresh in [0,.1,.2,.3,.4,.5,.6,.7]:
        #,0.01,.02,.05,.1
        for repeat in range(1):
            start = time.time()

#             Treatment = var_stub

#             var_list = [var_stub]
#             var_stub_list = [var_stub,]
#             mask = df_simp[var_stub].notnull()
#             wt_cols = []
#             min_features = 30
#             colname = var_stub
#             dependence_plots = False

#             alg = get_xgboost_alg(classification_problem=False)
#             alg.random_state = randint(0, 1000)
#             # alg.n_estimators=1000
#             title = var_stub

#             (explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
#                 xgboost_run(subdir=colname,dataset=df_simp[mask].drop(wt_cols,axis=1),
#                         var_list=var_list,var_stub_list=var_stub_list,
#                         use_specific_weights=None,
#                         min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
#                         skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
#                         title = title)

#             dist = pd.DataFrame(shap_values).abs().mean(axis=0).sort_values(ascending=False)
#             print( train_columns[dist[dist>=threshold].index].shape,df_simp.shape )

#             df = df_simp[train_columns[dist[dist>=threshold].index]][geomask]
#             df = df.fillna(df.mean())
            corrs = df_simp.drop(var_stub,axis=1).corrwith(df_simp[var_stub])
            df = df_simp[corrs[corrs.abs()>=corr_thresh].index]

            (BES_decomp, comp_labels, comp_dict) = dim_red(df, n_components=n_comp,red_type="Factor Analysis",show_first_x_comps=14)

            Y = df_simp[var_stub]
            X = BES_decomp.loc[:,0:n_comp]
            X = sm.add_constant(X)
            model = sm.OLS(Y,X)
            results = model.fit()
            display(results.summary())

            ind = results_df.index.max()
            if pd.isnull(ind):
                ind = 0
            else:
                ind = ind+1
            end = time.time()
    #         print(end - start)            
            results_df.loc[ind,["duration","n_components","variables","corr_thresh","repeats","results"]] = [end-start,n_comp,
                                                                                                           df.columns,
                                                                                                    corr_thresh,repeat,results]

In [ ]:
results_df["rsquared"] = results_df["results"].apply(lambda x: x.rsquared)

In [ ]:
results_df.groupby(['threshold','n_components']).mean()

In [ ]:
results_df.groupby(['threshold','n_components'])['repeats'].sum()

In [ ]:
%%time
var_stub = "Con17_elec"
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols+['Con17'],axis=1))[geomask]
df_simp[var_stub] = BES_census_data[var_stub][geomask].values


# results_list = []
# n_components=5
for n_comp in [6,7,8,9]:
    for threshold in [0.0,0.005,.01,.02,.05,.1,.2]:
        #,0.01,.02,.05,.1
        for repeat in range(20):
            start = time.time()

            Treatment = var_stub

            var_list = [var_stub]
            var_stub_list = [var_stub,]
            mask = df_simp[var_stub].notnull()
            wt_cols = []
            min_features = 30
            colname = var_stub
            dependence_plots = False

            alg = get_xgboost_alg(classification_problem=False)
            alg.random_state = randint(0, 1000)
            # alg.n_estimators=1000
            title = var_stub

            (explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
                xgboost_run(subdir=colname,dataset=df_simp[mask].drop(wt_cols,axis=1),
                        var_list=var_list,var_stub_list=var_stub_list,
                        use_specific_weights=None,
                        min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
                        skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
                        title = title)

            dist = pd.DataFrame(shap_values).abs().mean(axis=0).sort_values(ascending=False)
            print( train_columns[dist[dist>=threshold].index].shape,df_simp.shape )

            df = df_simp[train_columns[dist[dist>=threshold].index]][geomask]
            df = df.fillna(df.mean())

            (BES_decomp, comp_labels, comp_dict) = dim_red(df, n_components=n_comp,red_type="Factor Analysis",show_first_x_comps=14)

            Y = df_simp[var_stub]
            X = BES_decomp.loc[:,0:n_comp]
            X = sm.add_constant(X)
            model = sm.OLS(Y,X)
            results = model.fit()
            display(results.summary())

            ind = results_df.index.max()
            if pd.isnull(ind):
                ind = 0
            else:
                ind = ind+1
            end = time.time()
    #         print(end - start)            
            results_df.loc[ind,["duration","n_components","variables","threshold","repeats","results"]] = [end-start,n_comp,
                                                                                                           train_columns[dist[dist>=threshold].index],
                                                                                                    threshold,repeat,results]

In [ ]:
# results_df["n_components"] = results_df["n_components"].fillna(10)
results_df

In [ ]:
results_df.groupby(['threshold','n_components']).std()

In [ ]:
results_df

In [ ]:
# results_df["variables"].apply(lambda x: len(x)).groupby(results_df[['threshold','n_components']]).mean()

In [ ]:
df_simp.shape

In [ ]:
## add timing
# 5 mins per round




In [ ]:
df_simp = BES_constituency_aggregates.drop(['pano','constituency_name','Con17'],axis=1)\
    .drop(search(BES_constituency_aggregates,"|".join(["winConstituency","profile_past","ashcroft","partyContact",
                                                       "partyId","localElection","localTurnout","^map",
                                                       "generalElectionVote","euroElectionVote",
                                                       "^prefer[^M]","ptv"])).index,axis=1)
df_simp["Con17_elec"] = BES_census_data["Con17_elec"].values

var_stub = "Con17_elec"
Treatment = var_stub

var_list = [var_stub]
var_stub_list = [var_stub,]
mask = df_simp[var_stub].notnull()
wt_cols = []
min_features = 30
colname = "Con17_elec"
dependence_plots = False

alg = get_xgboost_alg(classification_problem=False)
title = "Con17_elec"

(explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
    xgboost_run(subdir=colname,dataset=df_simp[mask].drop(wt_cols,axis=1),
            var_list=var_list,var_stub_list=var_stub_list,
            use_specific_weights=None,
            min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
            skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
            title = title)

In [ ]:
dist = pd.DataFrame(shap_values).abs().mean(axis=0).sort_values(ascending=False)
train_columns[dist[dist>=.1].index]

In [ ]:
# less shit than expected

In [ ]:
df_simp = BES_census_data[["Con17"]]
df_simp.loc[BES_census_data["Country"]!="Scotland",nomis_dump.columns] = nomis_dump.values

In [ ]:
# df_simp = nomis_dump
# df_simp["Con17"] = BES_census_data["Con17"].values

var_stub = "Con17"
Treatment = var_stub

var_list = [var_stub]
var_stub_list = [var_stub,]
mask = df_simp[var_stub].notnull()
wt_cols = []
min_features = 30
colname = "Con17"
dependence_plots = False

alg = get_xgboost_alg(classification_problem=False)
title = "Con17"

(explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
    xgboost_run(subdir=colname,dataset=df_simp[mask].drop(wt_cols,axis=1),
            var_list=var_list,var_stub_list=var_stub_list,
            use_specific_weights=None,
            min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
            skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
            title = title)

In [ ]:
df_simp = BES_census_data[["Con17"]]
df_simp.loc[BES_census_data["Country"]!="Scotland",nomis_dump.columns] = nomis_dump.values

In [ ]:
# df_simp = nomis_dump
# df_simp["Con17"] = BES_census_data["Con17"].values

var_stub = "Con17"
Treatment = var_stub

var_list = [var_stub]
var_stub_list = [var_stub,]
mask = df_simp[var_stub].notnull()
wt_cols = []
min_features = 30
colname = "Con17"
dependence_plots = False

alg = get_xgboost_alg(classification_problem=False)
title = "Con17"

(explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
    xgboost_run(subdir=colname,dataset=df_simp[mask].drop(wt_cols,axis=1),
            var_list=var_list,var_stub_list=var_stub_list,
            use_specific_weights=None,
            min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
            skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
            title = title)

In [ ]:
%%time
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")
df_simp = BES_census_data[["Con17"]]
df_simp.loc[BES_census_data["Country"]!="Scotland",nomis_dump.columns] = nomis_dump.values
df_simp = df_simp.drop("Con17",axis=1)
df_simp = pd.get_dummies(pd.concat([df_simp,
                     BES_constituency_aggregates.drop(['pano','constituency_name','Con17'],axis=1).reset_index().drop("ons_const_id",axis=1),
                     BES_census_data],axis=1)[geomask].drop(drop_cols,axis=1))


In [ ]:
df_simp.shape,df_simp["Con17"]

In [ ]:
var_stub = "Con17"
Treatment = var_stub

var_list = [var_stub]
var_stub_list = [var_stub,]
mask = df_simp[var_stub].notnull()
wt_cols = []
min_features = 30
colname = "Con17"
dependence_plots = False

alg = get_xgboost_alg(classification_problem=False)
title = "Con17"

(explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
    xgboost_run(subdir=colname,dataset=df_simp[mask].drop(wt_cols,axis=1),
            var_list=var_list,var_stub_list=var_stub_list,
            use_specific_weights=None,
            min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
            skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
            title = title)

In [ ]:
# dataset = merged[merged["Country"].isin(["England","Wales"])]

Y = df_simp['Con17']
X = df_simp.fillna(df_simp.mean()).drop('Con17',axis=1)
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit_regularized(method = 'sqrt_lasso',refit=True,zero_tol =.005)
results.params

In [ ]:
results.params.sort_values().head(50)

In [ ]:
results.params.sort_values().tail(50)

In [ ]:
# - Con/Lab/LD/Lib-Left vs Auth-Right
# - Include non-voting
# - Do 19/17/15/10/05 X
# - Zoom into areas X
# - Think about feature selection

In [ ]:
BES_census_data.columns[0:20]

# Con,UKIP,Brexit
# Lab,LD,SNP,PC,Green
# Other - should turnout include this or not???

In [ ]:
# BES_census_data.drop(BES_census_data.columns[-82:],axis=1,inplace=True)

In [ ]:
BES_census_data.columns[-50:]

In [ ]:
var_stub = "Con17"
title = "Con17_jk_vars"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1))
specific_vars = ["c11Deprived3","c11QualLevel4","c11Age60to64","CON%MAN_DIR_SEN","CON%SAL_SERV",
                 "Merseyside",
                 "speakWelsh",
                ]
specific_vars = df_simp.columns
window = BES_census_data["Region"].isin(['North East', 'North West', 'Yorkshire and The Humber',
#                                          'East Midlands','West Midlands'
                                        ])
window = None

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")

sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)
# sm_and_xgb_plot(var_stub,title,specific_vars,df_simp,geomask=None)

In [ ]:
%%time
var_stub = "Con17"
title = "Con17_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
# specific_vars = ["c11Deprived3","c11QualLevel4","c11Age60to64","CON%MAN_DIR_SEN","CON%SAL_SERV",
#                  "Merseyside",
#                  "speakWelsh",
#                 ]
specific_vars = list(df_simp.columns)
# window = BES_census_data["Region"].isin(['North East', 'North West', 'Yorkshire and The Humber',
#                                          'East Midlands','West Midlands'
#                                         ])
window = None

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")

sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)
# sm_and_xgb_plot(var_stub,title,specific_vars,df_simp,geomask=None)

In [ ]:
%%time
var_stub = "Lab17"
title = "Lab17_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
var_stub = "Lab15"
title = "Lab15_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner15"]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
var_stub = "Lab10"
title = "Lab10_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner10"]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
var_stub = "Lab05"
title = "Lab05_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner05"]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
BES_census_data["Winner19"].value_counts()

In [ ]:
%%time
var_stub = "LD17"
title = "LD17_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
# specific_vars = ["c11Deprived3","c11QualLevel4","c11Age60to64","CON%MAN_DIR_SEN","CON%SAL_SERV",
#                  "Merseyside",
#                  "speakWelsh",
#                 ]
specific_vars = list(df_simp.columns)
# window = BES_census_data["Region"].isin(['North East', 'North West', 'Yorkshire and The Humber',
#                                          'East Midlands','West Midlands'
#                                         ])
window = None

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")

sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)
# sm_and_xgb_plot(var_stub,title,specific_vars,df_simp,geomask=None)

In [ ]:
%%time
var_stub = "LD15"
title = "LD15_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner15"]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
var_stub = "LD10"
title = "LD10_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner10"]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
var_stub = "LD05"
title = "LD05_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner05"]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
var_stub = "UKIP17"
title = "UKIP17_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
# specific_vars = ["c11Deprived3","c11QualLevel4","c11Age60to64","CON%MAN_DIR_SEN","CON%SAL_SERV",
#                  "Merseyside",
#                  "speakWelsh",
#                 ]
specific_vars = list(df_simp.columns)
# window = BES_census_data["Region"].isin(['North East', 'North West', 'Yorkshire and The Humber',
#                                          'East Midlands','West Midlands'
#                                         ])
window = None

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")

sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)
# sm_and_xgb_plot(var_stub,title,specific_vars,df_simp,geomask=None)

In [ ]:
%%time
ge = "15"
var_stub = "UKIP"+ge
title = "UKIP"+ge+"_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
ge = "10"
var_stub = "UKIP"+ge
title = "UKIP"+ge+"_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
ge = "05"
var_stub = "UKIP"+ge
title = "UKIP"+ge+"_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
%%time
var_stub = "Con15"
title = "Con15_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
# specific_vars = ["c11Deprived3","c11QualLevel4","c11Age60to64","CON%MAN_DIR_SEN","CON%SAL_SERV",
#                  "Merseyside",
#                  "speakWelsh",
#                 ]
specific_vars = list(df_simp.columns)
# window = BES_census_data["Region"].isin(['North East', 'North West', 'Yorkshire and The Humber',
#                                          'East Midlands','West Midlands'
#                                         ])
window = None

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")

sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)
# sm_and_xgb_plot(var_stub,title,specific_vars,df_simp,geomask=None)

In [ ]:
%%time
var_stub = "Con10"
title = "Con10_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
# specific_vars = ["c11Deprived3","c11QualLevel4","c11Age60to64","CON%MAN_DIR_SEN","CON%SAL_SERV",
#                  "Merseyside",
#                  "speakWelsh",
#                 ]
specific_vars = list(df_simp.columns)
# window = BES_census_data["Region"].isin(['North East', 'North West', 'Yorkshire and The Humber',
#                                          'East Midlands','West Midlands'
#                                         ])
window = None

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")

sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)
# sm_and_xgb_plot(var_stub,title,specific_vars,df_simp,geomask=None)

In [ ]:
%%time
var_stub = "Con05"
title = "Con05_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
# specific_vars = ["c11Deprived3","c11QualLevel4","c11Age60to64","CON%MAN_DIR_SEN","CON%SAL_SERV",
#                  "Merseyside",
#                  "speakWelsh",
#                 ]
specific_vars = list(df_simp.columns)
# window = BES_census_data["Region"].isin(['North East', 'North West', 'Yorkshire and The Humber',
#                                          'East Midlands','West Midlands'
#                                         ])
window = None

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")

sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)
# sm_and_xgb_plot(var_stub,title,specific_vars,df_simp,geomask=None)

In [ ]:
########################

In [ ]:
# %%time
ge = "17"
var_stub = "Con"+ge+"_elec"
title = "Con"+ge+"_elec_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
# %%time
ge = "17"
var_stub = "Lab"+ge+"_elec"
title = "Lab"+ge+"_elec_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
# %%time
ge = "17"
var_stub = "DNV"+ge+"_elec"
title = "DNV"+ge+"_elec_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
# %%timeDNV&OTH17_elec
# nope -> worse!
ge = "17"
var_stub = "DNV&OTH"+ge+"_elec"
title = "DNV&OTH"+ge+"_elec_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
# %%timeAuth_Right17_elec
ge = "17"
var_stub = "Auth_Right"+ge+"_elec"
title = "Auth_Right"+ge+"_elec_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
# %%timeAuth_Right17_elec
ge = "17"
var_stub = "Con"+ge+"_elec"
title = "Con"+ge+"_elec_kitchen_sink_england"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]=="England")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
# %%timeAuth_Right17_elec
ge = "17"
var_stub = "Auth_Right"+ge+"_elec"
title = "Auth_Right"+ge+"_elec_kitchen_sink_england"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]=="England")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
# %%timeAuth_Right17_elec
ge = "17"
var_stub = "Lib_Left"+ge+"_elec"
title = "Lib_Left"+ge+"_elec_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]=="England")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
# %%timeAuth_Right17_elec
ge = "17"
var_stub = "Lab"+ge+"_elec"
title = "Lab"+ge+"_elec_kitchen_sink"
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols,axis=1)).drop("Con17",axis=1)
specific_vars = list(df_simp.columns)
window = None
geomask = (BES_census_data["Country"]=="England")&(BES_census_data["Winner"+ge]!="Speaker")
sm_and_xgb_plot(var_stub,title,specific_vars,df_simp=df_simp,geomask=geomask,window=window)

In [ ]:
## confusing pattern with Lib_left/Lab r^2 in England&Wales/England
## opposite patternw with Right_auth/Con r^ in England&Wales/England

In [ ]:
## TODO
## option to suppress sm/geo plots for easy topline stuff
## feature selection
## -> try to get top X from xgboost
## -> factor analysis
#### (what about factor analysis/pca weighted by xgboost feature importance?)
## how about trying to fit against Con/X?% *simultaneously for 05/10/15/17(/19)
## tsne on top X factors?

In [ ]:
list(range(10))

In [ ]:
%%time
from sklearn import manifold, datasets
# perplexities = [5, 30, 50, 100]
perplexity = 5
n_components = 2
tsne = manifold.TSNE(
    n_components=n_components,
    init="random",
    random_state=0,
    perplexity=perplexity,
    learning_rate =100,
    n_iter=300,
)
Y = tsne.fit_transform(BES_decomp[[0, 1, 2, 4, 5, 6, 7, 8, ]])
palette = {'Conservative':"b", 'Labour':"r", 'Liberal Democrat':"orange", 'Plaid Cymru':"olive",
                  'Green':"g", 'Speaker':'grey', 'Scottish National Party':'y'}
plt.figure(figsize =(8,6));
sns.scatterplot(x=Y[:,0],y=Y[:,1],
                hue=BES_census_data["Winner19"].loc[BES_decomp.index],
                palette=palette)

In [ ]:
import umap

In [ ]:
u.min_dist

In [ ]:
u = umap.UMAP()

In [ ]:
reducer = umap.UMAP(n_neighbors=2, # def 15
                    min_dist=.1,n_components=2,metric="euclidean")
embedding = reducer.fit_transform(BES_decomp[[0, 1, 2, 4, 5, 6, 7, 8, ]])

plt.figure(figsize =(8,6));
sns.scatterplot(x=embedding[:,0],y=embedding[:,1],
                hue=BES_census_data["Winner19"].loc[BES_decomp.index],
                palette=palette)
# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
from umap.parametric_umap import ParametricUMAP
embedder = ParametricUMAP()
embedding = embedder.fit_transform(BES_decomp[[0, 1, 2, 4, 5, 6, 7, 8, ]])
plt.figure(figsize =(8,6));
sns.scatterplot(x=embedding[:,0],y=embedding[:,1],
                hue=BES_census_data["Winner19"].loc[BES_decomp.index],
                palette=palette)

In [ ]:
# plt.figure(figsize =(8,6));
# sns.scatterplot(x=BES_decomp[0],y=BES_decomp[1],
#                 hue=BES_census_data["Winner19"].loc[BES_decomp.index],
#                 palette=palette)

In [ ]:
## try PCA instead (more orth)
## try better FA!

In [ ]:
#             Y = df_simp[var_stub]
#             X = BES_decomp.loc[:,0:n_comp]
#             X = sm.add_constant(X)
#             model = sm.OLS(Y,X)
#             results = model.fit()
#             display(results.summary())

In [ ]:
### we want an okayish baseline for parameters

### we want a mixture of combos for Coalitions and Countries
# thresh =.1, n_components =10

# Countries -> Britain/England&Wales/Scotland/England/Wales/regional combos??
# Con/Auth-Right/Lab/Lib-Left/Lab+LD/Lab+LD+Green

# ~10 -> say we want an hour/hour-and-a-half
# 6-9 mins each

In [ ]:
%%time

geomask = (BES_census_data["Country"]!="Scotland")&(BES_census_data["Winner17"]!="Speaker")
df_simp = pd.get_dummies(BES_census_data.drop(drop_cols+['Con17'],axis=1))[geomask]
df_simp[var_stub] = BES_census_data[var_stub][geomask].values
var_stub = "Con17_elec"

# results_list = []
# n_components=5
for n_comp in [5,10]:
    for threshold in [0,.1,.2,.3,.4,.5,.6,.7]:
        #,0.01,.02,.05,.1
        for repeat in range(1):
            start = time.time()

#             Treatment = var_stub

#             var_list = [var_stub]
#             var_stub_list = [var_stub,]
#             mask = df_simp[var_stub].notnull()
#             wt_cols = []
#             min_features = 30
#             colname = var_stub
#             dependence_plots = False

#             alg = get_xgboost_alg(classification_problem=False)
#             alg.random_state = randint(0, 1000)
#             # alg.n_estimators=1000
#             title = var_stub

#             (explainer, shap_values, train_columns, train_index, alg,output_subfolder)=\
#                 xgboost_run(subdir=colname,dataset=df_simp[mask].drop(wt_cols,axis=1),
#                         var_list=var_list,var_stub_list=var_stub_list,
#                         use_specific_weights=None,
#                         min_features = min(df_simp.shape[1]-1,min_features),verbosity=0,
#                         skip_bar_plot=True,dependence_plots=dependence_plots,alg=alg,eval_metric=eval_metric,                    
#                         title = title)

#             dist = pd.DataFrame(shap_values).abs().mean(axis=0).sort_values(ascending=False)
#             print( train_columns[dist[dist>=threshold].index].shape,df_simp.shape )

#             df = df_simp[train_columns[dist[dist>=threshold].index]][geomask]
#             df = df.fillna(df.mean())
            corrs = df_simp.drop(var_stub,axis=1).corrwith(df_simp[var_stub])
            df = df_simp[corrs[corrs.abs()>=corr_thresh].index]

            (BES_decomp, comp_labels, comp_dict) = dim_red(df, n_components=n_comp,red_type="Factor Analysis",show_first_x_comps=14)

            Y = df_simp[var_stub]
            X = BES_decomp.loc[:,0:n_comp]
            X = sm.add_constant(X)
            model = sm.OLS(Y,X)
            results = model.fit()
            display(results.summary())

            ind = results_df.index.max()
            if pd.isnull(ind):
                ind = 0
            else:
                ind = ind+1
            end = time.time()
    #         print(end - start)            
            results_df.loc[ind,["duration","n_components","variables","corr_thresh","repeats","results"]] = [end-start,n_comp,
                                                                                                           df.columns,
                                                                                                    corr_thresh,repeat,results]